In [4]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install scikit-learn


In [2]:
!pip install xgboost

   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
    --------------------------------------- 1.3/56.8 MB 8.4 MB/s eta 0:00:07
   -- ------------------------------------- 3.4/56.8 MB 8.4 MB/s eta 0:00:07
   --- ------------------------------------ 5.2/56.8 MB 8.8 MB/s eta 0:00:06
   ----- ---------------------------------- 7.3/56.8 MB 9.2 MB/s eta 0:00:06
   ------ --------------------------------- 9.7/56.8 MB 9.4 MB/s eta 0:00:05
   -------- ------------------------------- 12.1/56.8 MB 9.6 MB/s eta 0:00:05
   --------- ------------------------------ 13.9/56.8 MB 9.6 MB/s eta 0:00:05
   ----------- ---------------------------- 16.0/56.8 MB 9.5 MB/s eta 0:00:05
   ------------ --------------------------- 18.1/56.8 MB 9.5 MB/s eta 0:00:05
   -------------- ------------------------- 20.2/56.8 MB 9.5 MB/s eta 0:00:04
   --------------- ------------------------ 22.3/56.8 MB 9.6 MB/s eta 0:00:04
   ----------------- ---------------------- 24.6/56.8 MB 9.7 MB/s eta 0:00:04

In [5]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [7]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 42,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [8]:
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.9560439560439561,
  'support': 270.0},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30.0},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300.0},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.9159951159951161,
  'support': 300.0}}

In [10]:
!pip install mlflow

  Using cached mlflow-3.4.0-py3-none-any.whl.metadata (30 kB)
  Using cached mlflow_skinny-3.4.0-py3-none-any.whl.metadata (31 kB)
  Using cached mlflow_tracing-3.4.0-py3-none-any.whl.metadata (19 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached fastmcp-2.12.4-py3-none-any.whl.metadata (19 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached waitress-3.0.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached databricks_sdk-0.67.0-py3-none-any.whl.metadata (39 kB)
  Using cached opentelemetry_api-1.37.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_proto-1.37.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.37.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached authlib-1.6.4-py2.py3-none-any.whl.metadata (9.8 kB)
  Using cached cyclopts-3.24.0-py3-none-any.whl.metadata (11 kB)
  Using cached exceptiongroup-1.3.0-py3-no

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.32.1 which is incompatible.
streamlit 1.37.1 requires rich<14,>=10.14.0, but you have rich 14.1.0 which is incompatible.


In [11]:
import mlflow

In [13]:
mlflow.set_experiment("1st Experiment")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")
#mlflow.set_tracking_uri("http://localhost:5000")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'recall_class_0': report_dict['0']['recall'],
        'recall_class_1': report_dict['1']['recall'],
        'f1_score_macro': report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr, "Logistic Regression")  

2025/10/01 18:31:41 INFO mlflow.tracking.fluent: Experiment with name '1st Experiment' does not exist. Creating a new experiment.
2025/10/01 18:31:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/01 18:32:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run luxuriant-mare-405 at: http://127.0.0.1:5000/#/experiments/287540816721055399/runs/8658cf9cbbf14260a35c0baf652348c4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/287540816721055399
